In [1]:
import sys
sys.path.append(r'./script/')
from ECMpy_function import *
import copy
import math

# μ=0.46 sub=5.5 compare pathway producing lysine rates between 0.1 and 0.46

In [2]:
json_model_file="./model/iCW773_irr_enz_constraint_adj_PDH.json"
enz_model=get_enzyme_constraint_model(json_model_file)
# reaction_kcat_MW_file="./analysis/reaction_kcat_MW.csv"
objlist = []
glclist = []
exlist = list(np.linspace(0.1,0.46,2))
exlistn = []
for i in exlist:
    i = format(i,'.2f')
    exlistn.append(float(i))
# print(exlistn)
concentration = 5.5
obj='EX_lys_L_e' # EX_lys_L_e
enzcost = pd.DataFrame()

for i in exlistn:
    cond = i

    enz_model.reactions.get_by_id('EX_glc_e').bounds=(-concentration,0)#EX_glc_e EX_fru_e EX_sucr_e EX_inost_e EX_lac_D_e EX_ac_e
    enz_model.reactions.get_by_id('EX_glc_e_reverse').bounds=(0,0)
    # enz_model.reactions.get_by_id('ATPM').bounds=(6.86,6.86)
    enz_model.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(cond,cond)
    enz_model.objective=obj
    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    glclist.append(str(i))
    objlist.append(enz_model_pfba_solution.fluxes[obj])
    
    print(-enz_model_pfba_solution.fluxes['EX_glc_e'])
    print(enz_model_pfba_solution.fluxes['EX_ac_e'])
    print(enz_model_pfba_solution.fluxes['CG_biomass_cgl_ATCC13032'])
    fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba_%.2g.csv'%(obj,cond,concentration)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_model_file)
    enzcost['id']= enz_model_pfba_solution.index
    enzcost.set_index('id',inplace=True)
    enzcost['E_μ%s'%(cond)] = enz_model_pfba_solution['E']
    
    # print(enzcost['E_%s'%(cond)] )
    print(np.sum(enz_model_pfba_solution['E']))
    print("        ")
ectest = pd.DataFrame({'ec773':glclist,'obj':objlist})
print(ectest)



folddf = pd.DataFrame()
for row in enzcost.itertuples():
    if re.search(row.Index,'DAPDC_num2|DAPDH|DHDPS|ASPK|PYRC|TKT2|RPE|TKT1|TALA|GND|G6PDH2r|PGL|ENO|PGK|PDH|ICDHyr|ACONTa|ACONTb|CS_num1'):
        # print(enzcost.loc(row.Index,'E_μ0.1'))
        folddf.loc[row.Index,'E_μ=0.1'] = enzcost.loc[row.Index,'E_μ0.1']
        folddf.loc[row.Index,'E_μ=0.46'] = enzcost.loc[row.Index,'E_μ0.46']
folddf['log2_foldchange'] = folddf['E_μ=0.1']/folddf['E_μ=0.46']
foldlist = []
for eachf in folddf['log2_foldchange']:
    eachf = math.log2(eachf)
    foldlist.append(eachf)    
folddf['log2_foldchange'] = foldlist
folddf = folddf.sort_values(by='log2_foldchange',ascending=False)
folddf.to_csv('./analysis/enzcost_foldchange.csv')
folddf



0.12
5.5
0.0
0.1
Enzyme cost total is:0.12000000000000109
0.12000000000000109
        
5.5
1.1332697883576075
0.46
Enzyme cost total is:0.12000000000000002
0.12000000000000002
        
  ec773       obj
0   0.1  3.645972
1  0.46  0.145764


,E_μ=0.1,E_μ=0.46,log2_foldchange
DAPDC_num2,0.002627,1.719283e-04,3.933409
DAPDH,0.000004,2.657299e-07,3.808982
DHDPS,0.000599,4.271488e-05,3.808982
ASPK,0.006895,1.121867e-03,2.619702
PYRC,0.003908,8.637034e-04,2.177659
TKT2,0.000144,6.857212e-05,1.066334
RPE,0.000021,1.049781e-05,0.992251
TKT1,0.000001,6.943411e-07,0.922623
TALA,0.000145,7.668502e-05,0.922623
PGL,0.003047,1.753960e-03,0.796957
